## **<center><font style="color:rgb(100,109,254)">Examination Answer Sheet Marks Entrance Using OCR</font> </center>**
### **<center><font style="color:rgb(100,109,254)">Machine Learning Project</font> </center>**

In [1]:
import cv2
print(cv2.__version__)
import pytesseract

pytesseract.pytesseract.tesseract_cmd = "C:\\Program Files\\Tesseract-OCR\\tesseract.exe"


4.6.0


In [4]:
import os
import numpy as np

imgq = cv2.imread("q1.jpeg")
h,w,c = imgq.shape
imgq = cv2.resize(imgq, (w//3, h//3))

orb = cv2.ORB_create(1000)
kp1, des1 = orb.detectAndCompute(imgq,None)
# imgKp1 = cv2.drawKeypoints(imgq, kp1, None)
per = 25

roi = [[(236, 294), (948, 388), 'text', 'StudentName'],
        [(216, 372), (692, 450), 'text', 'RegistrationNumber'],
        [(794, 382), (1254, 458), 'text', 'Subject'],
        [(1060, 448), (1260, 520), 'text', 'class'],
        [(1066, 646), (1250, 720), 'text', 'totalMarks'],
        [(1068, 864), (1256, 938), 'text', 'obtainedMarks']]

path = 'UserForms'
myImageList = os.listdir(path)
print(myImageList)

for j, y in enumerate(myImageList):
    img = cv2.imread(path+"/"+y)
    img = cv2.resize(img, (w//3, h//3))
    # img = cv2.resize(img, (w,h))
    # cv2.imshow("ss", img)
    kp2, des2 =  orb.detectAndCompute(img,None)
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    matches = bf.match(des2, des1)
    # matches.sort(key= lambda x: x.distance)
    good = matches[:int(len(matches)*(per/100))]

    imgMatch = cv2.drawMatches(img, kp2, imgq, kp1, good[:100], None, flags=2)
    # imgMatch = cv2.resize(imgMatch, (w,h))
    # cv2.imshow(y, imgMatch)

    srcPonits = np.float32([kp2[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
    dstPonits = np.float32([kp1[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

    M, _ = cv2.findHomography(srcPonits, dstPonits, cv2.RANSAC, 5.0)
    imgScan = cv2.warpPerspective(img, M, (w//3, h//3))
    imgScan = cv2.resize(imgScan, (w,h))
    cv2.imshow(y, imgScan)

    imgShow = imgScan.copy()
    imgMask = np.zeros_like(imgScan)

    for x,r in enumerate(roi):

        cv2.rectangle(imgMask, (r[0][0], r[0][1]), (r[1][0], r[1][1]), (0,255,0), cv2.FILLED)
        imgShow = cv2.addWeighted(imgShow, 0.99, imgMask, 0.1, 0)

        imgCrop = imgScan[r[0][1]:r[1][1], r[0][0]:r[1][0]]
        cv2.imshow(str(x), imgCrop)

        if r[2] == 'text':
            confg = r'--oem 3 --psm 6 outputbase digits'
            print(f"{r[3]} : {pytesseract.image_to_string(imgCrop, config=confg)}")

    imgShow = cv2.resize(imgShow, (w,h))
    cv2.imshow(y+"2", imgShow)





# cv2.imshow("Image KP1", imgKp1)
cv2.imshow("Image", imgq)
cv2.waitKey()
cv2.destroyAllWindows()

['11.jpeg', '22.jpeg']
StudentName : 0

RegistrationNumber : 202203033

Subject : -

class : .-

totalMarks : 
obtainedMarks : 23

StudentName : 
RegistrationNumber : 202202033

Subject : 
class : 8.-

totalMarks : 2

obtainedMarks : 238



In [3]:
cv2.destroyAllWindows()